In [1]:
from data.PatchesProviders import RasterPatchProvider, MultipleRasterPatchProvider, MetaPatchProvider
from data.Datasets import PatchesDataset

In [37]:
import pandas as pd
import numpy as np

In [2]:
datadir = 'data/full_data/'
p = MultipleRasterPatchProvider(
    datadir + 'EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/',
    size = 4, normalize=True, select=['bio1','bio11']
)

In [3]:
data1 = PatchesDataset(datadir + 'Presence_only_occurrences/Presences_only_train.csv', p, one_provider=True)

In [6]:
print(len(data1.species), data1.species[0:20])

10038 [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


In [7]:
data2 = PatchesDataset(datadir + 'Presence_Absence_surveys/Presences_Absences_train.csv', p, one_provider=True)

In [9]:
print(len(data2.species), data2.species[0:20])

2174 [  1   5  10  11  24  28  33  37  38  55  56  76  95  99 101 103 114 118
 119 121]


In [15]:
data_po = PatchesDataset(datadir + 'Presence_only_occurrences/Presences_only_train.csv', p, one_provider=True, species=data2.species)

In [16]:
print(len(data_po.species), data_po.species[0:20])

2174 [  1   5  10  11  24  28  33  37  38  55  56  76  95  99 101 103 114 118
 119 121]


In [44]:
df = pd.read_csv(datadir + 'Presence_only_occurrences/Presences_only_train.csv', sep=";", header='infer', low_memory=False)
species = data2.species
df = df[df['speciesId'].isin(species)]
items = df[['lat','lon','patchID','dayOfYear'] + ['speciesId']]

In [45]:
df.shape

(3558526, 15)

In [46]:
items[items['patchID'] == 3018874]

,lat,lon,patchID,dayOfYear,speciesId


In [47]:
item = items.iloc[5][['lat','lon','patchID','dayOfYear']].to_dict()
item_species = items.query(' and '.join([f'{k} == {v}' for k, v in item.items()]))['speciesId'].values
item_species

array([5020])

In [48]:
labels = 1 * np.isin(species, item_species)
labels

array([0, 0, 0, ..., 0, 0, 0])

In [49]:
np.sum(labels)

1

In [ ]:
 item = self.items.iloc[index][self.item_columns].to_dict()
        print(item)

        item_species = self.items.query(
            ' and '.join([f'{k} == {v}' for k, v in item.items()])
        )[self.label_name].values
        labels = 1 * np.isin(self.species, item_species)

In [11]:
df = pd.read_csv(datadir + 'Presence_Absence_surveys/Presences_Absences_train.csv', sep=';')
df

,glcID,gbifID,observer,datasetName,date,dayOfYear,year,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,speciesId,patchID,timeSerieID
0,5140269,3.473385e+09,(IGN),Inventaire forestier IGN,2018-01-31,31,2018,7.300790,48.29273,4.120673e+06,2.801353e+06,1.0,6574,3018575,3980325
1,5279421,3.474159e+09,(IGN),Inventaire forestier IGN,2017-07-19,200,2017,1.483390,42.92599,3.625340e+06,2.241565e+06,1.0,6574,3011783,3980331
2,5058088,3.475680e+09,(IGN),Inventaire forestier IGN,2017-09-26,269,2017,6.739570,47.39633,4.074778e+06,2.703502e+06,1.0,6574,3018192,3980335
3,5146840,3.476162e+09,(IGN),Inventaire forestier IGN,2017-09-20,263,2017,4.587140,45.40615,3.897015e+06,2.492494e+06,1.0,6574,3015541,3980336
4,5255062,3.475124e+09,(IGN),Inventaire forestier IGN,2019-09-18,261,2019,6.955830,47.74829,4.092660e+06,2.741886e+06,1.0,6574,3018391,3980351
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85320,5320539,NaN,NaN,CBNMed,1621461600,140,2021,5.094570,44.18660,3.928254e+06,2.354689e+06,10.0,9456,76900,3993145
85321,5392608,NaN,NaN,CBNMed,1497909600,171,2017,5.781197,43.74345,3.980545e+06,2.302273e+06,10.0,9456,94452,3986894
85322,5353058,NaN,NaN,CBNMed,1621461600,140,2021,5.094620,44.18680,3.928259e+06,2.354711e+06,10.0,9456,76905,3993923
85323,5102872,2.907311e+09,"Shoesmith, J. Judith",Nat. plant monitoring UK,2017-06-07,158,2017,0.785642,51.70740,3.686016e+06,3.217623e+06,7.1,6081,3010687,3990777


In [14]:
df.groupby('speciesId').glcID.count()

speciesId
1          3
5         14
10         5
11        20
24       588
        ... 
10028      3
10031      1
10035     16
10038      1
10039     48
Name: glcID, Length: 2174, dtype: int64

In [13]:
occurrences = datadir + 'Presence_only_occurrences/Presences_only_train.csv'
label_name='speciesId'
item_columns=['lat','lon','patchID','dayOfYear']
df = pd.read_csv(occurrences, sep=";", header='infer', low_memory=False)

In [16]:
items = df[item_columns + [label_name]]
species = np.unique(df[label_name].values)
base_providers = p

In [20]:
# provider = MetaPatchProvider(base_providers)
base_providers

Rasters in folder:
band_names: ['bio1']
n_layers: 1
x_min: -32.266806146449994
y_min: 26.63319397795
x_resolution: 0.0083333333
y_resolution: 0.0083333333
n_rows: 5467
n_cols: 8143
--------------------------------------------------
band_names: ['bio11']
n_layers: 1
x_min: -32.266806146449994
y_min: 26.63319397795
x_resolution: 0.0083333333
y_resolution: 0.0083333333
n_rows: 5467
n_cols: 8143
--------------------------------------------------

In [25]:
len(base_providers)

2

In [21]:
nb_layers = sum([len(provider) for provider in base_providers])

TypeError: 'int' object is not subscriptable

In [ ]:
data[0]

In [20]:
index = 0
item = data.items.iloc[index][data.item_columns].to_dict()
print(item)

item_species = data.items.query(
    ' and '.join([f'{k} == {v}' for k, v in item.items()])
)[data.label_name].values
labels = 1 * np.isin(data.species, item_species)
print(item_species)
print(labels.sum())

{'lat': 51.77186, 'lon': -10.53904, 'patchID': 3018869.0, 'dayOfYear': 272.0}
[8075]
1


In [21]:
p[item]

array([2.84300000e+03, 2.83900000e+03, 8.70000000e+01, 2.80400000e+03,
       2.89100000e+03, 2.45400000e+03, 2.00999999e+00, 1.80000000e+01,
       5.73500000e+03, 4.40300000e+03, 3.57900000e+03, 6.82100000e+03,
       2.60000000e+02, 1.16800000e+03, 2.34000000e+03, 2.05380000e+04,
       2.81700000e+03, 2.87900000e+03, 2.84500000e+03])

In [14]:
p[item].shape

(2,)

In [4]:
import itertools
import logging
import math
import os
from abc import abstractmethod
from random import random

import matplotlib.pyplot as plt
import numpy as np
import pyproj
import rasterio
from PIL import Image
import pandas as pd

from pathlib import Path
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import rasterio

In [17]:
#RasterPatchProvider
data_path = 'data/sample_data/' # root path of the data
raster_path = data_path+'EnvironmentalRasters/Climate/BioClimatic_Average_1981-2010/bio1.tif'
size=128
spatial_noise=0
normalize=False
fill_zero_if_error=False
nan_value=0
transformer = None
name = os.path.basename(os.path.splitext(raster_path)[0])

In [18]:
with rasterio.open(raster_path) as src:
    # read the metadata of the file
    meta = src.meta
    meta.update(count=src.count) # update the count of the meta to match the number of layers
    print('META:', meta)

    # read the data from the raster
    data = src.read()
    print('SHAPE:', data.shape)

    # get the NoData value from the raster
    nodata_value = src.nodatavals
    print('NODATAVALUE:', nodata_value)

    # iterate through all the layers
    for i in range(src.count):
        print(i)
        # replace the NoData values with np.nan
        data = data.astype(float)
        data[i] = np.where(data[i] == nodata_value[i], np.nan, data[i])
        if normalize:
            data[i] = (data[i] - np.nanmean(data[i]))/np.nanstd(data[i])
        data[i] = np.where(np.isnan(data[i]), nan_value, data[i])
            
    nb_layers = src.count
    print('NB LAYERS:', nb_layers)

    x_min = src.bounds.left
    y_min = src.bounds.bottom
    x_resolution = src.res[0]
    y_resolution = src.res[1]
    n_rows = src.height
    n_cols = src.width
    crs = src.crs
    print(x_min, y_min, x_resolution, y_resolution, n_rows, n_cols, crs)
    if nb_layers > 1:
        bands_names = [name+'_'+str(i+1) for i in range(nb_layers)]
    else:
        bands_names = [name]
        
    epsg = crs.to_epsg()
    if epsg != 4326:
        # create a pyproj transformer object to convert lat, lon to EPSG:32738
        transformer = pyproj.Transformer.from_crs("epsg:4326", epsg, always_xy=True)


META: {'driver': 'GTiff', 'dtype': 'uint16', 'nodata': None, 'width': 8143, 'height': 5467, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0083333333, 0.0, -32.266806146449994,
       0.0, -0.0083333333, 72.19152712905)}
SHAPE: (1, 5467, 8143)
NODATAVALUE: (None,)
0
NB LAYERS: 1
-32.266806146449994 26.63319397795 0.0083333333 0.0083333333 5467 8143 EPSG:4326


In [19]:
#PatchesDataset
occurrences = Path(data_path+'Presence_only_occurrences/Presences_only_train_sample.csv')
transform = None
target_transform = None
id_name="glcID"
label_name="speciesId"
item_columns=['lat', 'lon', 'patchID']
df = pd.read_csv(occurrences, sep=";", header='infer', low_memory=False)
df.head()

,glcID,gbifID,observer,datasetName,date,dayOfYear,year,year_ecodatacube_quarter,lon,lat,x_EPSG3035,y_EPSG3035,geoUncertaintyInM,speciesId,patchID,timeSerieID
0,2522017,1.882957e+09,Bjørn Petter Løfall,NOR Species Observation,2018-06-17,168,2018,2.0,10.951540,59.25110,4375406.0,4017067.0,10.0,9343,5982155,2959445
1,915820,3.704105e+09,"Szokala,Daniel",Masaryk Univ Herbarium,2020-08-23,236,2020,3.0,22.678360,43.39697,5345300.0,2341095.0,10.0,2165,6920543,3900275
2,2204483,2.634620e+09,Mykyta Peregrym,iNaturalist RG,2020-06-01,153,2020,2.0,20.057780,47.91972,5070487.0,2807349.0,5.0,7722,6876987,3856654
3,1892353,2.894783e+09,Vejle Kommune,DEN Environmental Portal,2018-05-16,136,2018,2.0,9.337563,55.75870,4279387.0,3628455.0,3.0,6461,5547194,2523020
4,3377718,1.884870e+09,Bernt-Gunnar Østerkløft,NOR Species Observation,2018-07-18,199,2018,3.0,14.614180,67.38264,4520783.0,4924950.0,50.0,7287,6592021,3570234


In [39]:
items = df[item_columns].drop_duplicates().reset_index(drop=True)
items

,lat,lon,patchID
0,59.25110,10.951540,5982155
1,43.39697,22.678360,6920543
2,47.91972,20.057780,6876987
3,55.75870,9.337563,5547194
4,67.38264,14.614180,6592021
...,...,...,...
95,51.13202,17.369410,6808612
96,59.13283,9.546747,5598953
97,56.43098,16.439410,6749721
98,50.86739,9.702714,5641177


In [ ]:
occurrences['speciesId']

In [20]:
observation_ids = df[id_name].values
items = df[item_columns]
targets = df[label_name].values

In [21]:
#dataset get_item
index = 0
item = items.iloc[index].to_dict()
print('ITEM:', item)

target = targets[index]
print('TARGET:', target)

ITEM: {'lat': 59.2511, 'lon': 10.95154, 'patchID': 5982155.0}
TARGET: 9343


In [30]:
#patch = self.provider[item]
#provider get_item
lon, lat = (item['lon'], item['lat'])
print(lon, lat)

x = int(n_rows - (lat - y_min) / y_resolution)
y = int((lon - x_min) / x_resolution)
print(x, y)

10.95154 59.2511
1552 5186


In [24]:
data.shape

(1, 5467, 8143)

In [32]:
patch_size = 1
patch_data = [data[i, x, y] for i in range(nb_layers)]
patch_data

[2803.0]

In [35]:
patch_size = 6
patch_data = [data[i, x - (patch_size // 2): x + (patch_size // 2), 
                   y - (patch_size // 2): y + (patch_size // 2)] 
                   for i in range(nb_layers)]
patch_data

[array([[2805., 2803., 2804., 2803., 2803., 2801.],
        [2804., 2804., 2804., 2803., 2802., 2801.],
        [2805., 2804., 2804., 2803., 2802., 2802.],
        [2806., 2805., 2803., 2803., 2802., 2804.],
        [2806., 2805., 2803., 2803., 2804., 2806.],
        [2806., 2805., 2804., 2804., 2805., 2806.]])]

In [77]:
df_full_raw = pd.read_csv('data/full_data/Presence_only_occurrences/Presences_only_train.csv', sep=';',header='infer',low_memory=True)

In [41]:
df_full_raw.shape

(4908319, 15)

In [80]:
item_columns=['lat','lon','patchID','dayOfYear']
label_name="speciesId"
df_full = df_full_raw.drop_duplicates(item_columns + [label_name])
print(df_full.shape)
items = df_full[item_columns + [label_name]]
print(items.shape)

(4905577, 15)
(4905577, 5)


In [82]:
items.head()

,lat,lon,patchID,dayOfYear,speciesId
0,51.77186,-10.53904,3018869,272,8075
1,51.77226,-10.53875,3018870,234,8075
2,52.11013,-10.47597,3018871,191,6477
3,52.15074,-10.47555,3018872,183,5783
4,52.10978,-10.47497,3018873,158,9011


In [50]:
label_name="speciesId"
targets = df_full[label_name].values
targets.shape

(4007514,)

In [51]:
targets[0:5]

array([8075, 8075, 6477, 5783, 9011])

In [58]:
np.unique(targets).shape#[0:5]#.sort_values().unique()#[0:5]

(9826,)

In [135]:
index = 604
item = items.iloc[index][item_columns].to_dict()
item

{'lat': 52.05827, 'lon': -9.932635, 'patchID': 3019469.0, 'dayOfYear': 365.0}

In [139]:
query_str = ' and '.join([f'{k} == {v}' for k, v in item.items()])
filtered_df = items.query(query_str)
species = filtered_df['speciesId'].values
species

array([ 277,  665, 3397, 3569, 4284, 5685, 5783, 6588, 8058, 9020, 9496,
       9595, 9747])

In [ ]:
labels = 1 * np.isin(self.species, item_species)

In [132]:
tmp = items.groupby(['patchID','dayOfYear']).lat.count()
tmp[tmp != 1]

patchID  dayOfYear
3019203  183           2
3019217  146           2
3019469  365          13
3019473  295           2
3019715  2             4
                      ..
6999952  61            2
7000005  63            2
7000044  311           2
7000124  62            8
7000159  64            2
Name: lat, Length: 268284, dtype: int64

In [125]:
print(items.shape)
for col in item_columns:
    print(items[col] == item[col])
    #print(col, filtered_items.shape)  


(4905577, 5)
0          False
1          False
2          False
3          False
4          False
           ...  
4908314    False
4908315    False
4908316    False
4908317    False
4908318    False
Name: lat, Length: 4905577, dtype: bool
0          False
1          False
2          False
3          False
4          False
           ...  
4908314    False
4908315    False
4908316    False
4908317    False
4908318    False
Name: lon, Length: 4905577, dtype: bool
0          False
1          False
2          False
3          False
4          False
           ...  
4908314    False
4908315    False
4908316    False
4908317    False
4908318    False
Name: patchID, Length: 4905577, dtype: bool
0          False
1           True
2          False
3          False
4          False
           ...  
4908314    False
4908315    False
4908316    False
4908317    False
4908318    False
Name: dayOfYear, Length: 4905577, dtype: bool


In [121]:
items[
    (items['patchID'] == item['patchID']) & (items['dayOfYear'] == item['dayOfYear']) & 
    (items['lat'] == item['lat']) & (items['lon'] == item['lon']) 
]#['speciesId'].values

,lat,lon,patchID,dayOfYear,speciesId
13,52.13511,-10.46012,3018882,234,4287


In [93]:
for i in items.iterrows():
    # print(i[0])
    # print([i[1][k] == v for k,v in item.items()])
    if all([i[1][k] == v for k,v in item.items()]): 
        print('SPECIES:', i[1][label_name])
        print([f"{k}: {i[1][k]}" for k, v in item.items()])
    # if i[0] > 3: break

SPECIES: 8075.0
['lat: 51.77186', 'lon: -10.53904', 'patchID: 3018869.0', 'dayOfYear: 272.0']


KeyboardInterrupt: 

In [20]:
#get item

BoundingBox(left=-32.266806146449994, bottom=26.63319397795, right=35.591526915450004, top=72.19152712905)